# Generacion de nuestro propio modelo de W2V a partir de los datos de twitter y analisis de similitud entre usuarios a partir de estos vectores

#### lo que hacemos es utilizar una red neuronal para interrelacionar las palabras de un texto. Como?

usando un autoencoder al que luego quitamos la ultima capa, esto nos da los vectores de cada palabra una red neuronal con n entradas (palabras unicas en el texto), m neuronas en capa intermedia, y o neuronas en la salida. el sistema intenta predecir qué palabra vendrá despues de cada palabra de entrada. siendo la palabra un vector de 1Xn.

el sistema aprenderá que palabras con vectores parecidos estan inter relacionadas (pertenecen al mismo tema)

al cortar la capa final, lo que nos queda es una matriz de datos con la probabilidad de aparicion de una palabra para las demas. eso es un vector

# Load Twitter Data

In [1]:
import gensim
import pandas as pd
import pyodbc
import re
from gensim.models import Word2Vec
import numpy as np
from tqdm import tqdm_notebook
from scipy.spatial import distance
from scipy.spatial.distance import mahalanobis
import seaborn as sns 
import matplotlib.pyplot as plt 
from scipy.stats import norm

In [2]:
import configLocal
server = configLocal.server
database = configLocal.database
username = configLocal.username
password = configLocal.password
cnxn = configLocal.cnxn
cursor = cnxn.cursor()

data = pd.read_sql_query("select msg_id, msg_users_id, msg_timestamp, msg_text from dbo.msg", cnxn)

In [3]:
#parse database to dict
corpus = {}
for i in range(data.shape[0]):
    if data.iloc[i]["msg_users_id"] in corpus:
        corpus[data.iloc[i]["msg_users_id"]].append(re.sub(r'http\S+', '', data.iloc[i]["msg_text"], flags=re.MULTILINE))
    else:
        corpus[data.iloc[i]["msg_users_id"]] = [re.sub(r'http\S+', '', data.iloc[i]["msg_text"], flags=re.MULTILINE)]

print(type(corpus), len(corpus))

for i in corpus.keys():
    corpus[i] = gensim.utils.simple_preprocess("".join(corpus[i]).encode('utf-8'), deacc=True, min_len=3)

<class 'dict'> 7157


carga del giga modelo

In [ ]:
#now, lets load da big boi model

from numpy import  zeros, float32 as REAL
from gensim.models import keyedvectors
import codecs

# this function was build using code excerpts from:
# https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/models/keyedvectors.py
def load_vectors_from_csv(fname,vocab_size=973265,vector_size=100):
    print("Loading vectors from file:",fname)
    result=keyedvectors.KeyedVectors(vector_size=100)
    result.syn0 = zeros((vocab_size, vector_size), dtype=REAL)
    result.vector_size=vector_size
    counts=None   
    def add_word(word, weights):
        word_id = len(result.vocab)
        if word in result.vocab:
            print("duplicate word '%s' in %s, ignoring all but first", word, fname)
            return
        if counts is None:
            # most common scenario: no vocab file given. just make up some bogus counts, in descending order
            result.vocab[word] = keyedvectors.Vocab(index=word_id, count=vocab_size - word_id)
        elif word in counts:
            # use count from the vocab file
            result.vocab[word] = keyedvectors.Vocab(index=word_id, count=counts[word])
        else:
            # vocab file given, but word is missing -- set count to None (TODO: or raise?)
            print("vocabulary file is incomplete: '%s' is missing", word)
            result.vocab[word] = keyedvectors.Vocab(index=word_id, count=None)
        result.syn0[word_id] = weights
        result.index2word.append(word)   
    file=codecs.open(fname,"r","utf-8")
    i=0
    for line in file:
        i+=1
        if i==1: #ommit header
            continue
        parts=line.strip().split(",")
        word,weights=parts[1],[REAL(x) for x in parts[2:]]
        add_word(word,weights)
        if i%100000==0:
            print(i,"word vectors loaded so far ...")
    file.close()
    print(i-1,"word vectors loaded!")
    return result
    

    
model=load_vectors_from_csv("data/WORD2VEC-Twitter-Espa_ol_para_Latinoam_rica__Espa_a_y_Estados_Unidos2.csv")

--------------

# Creacion de modelo y visualizacion de temas con tensorboard (en construccion)


## preprocesing twitter data

In [7]:
from tensorflow.contrib.tensorboard.plugins import projector
from sklearn.manifold import TSNE
from collections import Counter
from six.moves import cPickle
import gensim.models.word2vec as w2v
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import multiprocessing
import os
import sys
import io
import re
import json
from tqdm import tqdm_notebook as tqdm
from string import punctuation
from nltk.corpus import stopwords

cachedStopWords = stopwords.words("spanish")

In [8]:
#generar una lista con los tweets sucios
df = [x for x in data.msg_text]

In [9]:
#funcion de limpieza:
def process_raw_data(input_file):
    url_match = "(https?:\/\/[0-9a-zA-Z\-\_]+\.[\-\_0-9a-zA-Z]+\.?[0-9a-zA-Z\-\_]*\/?.*)"
    name_match = "\@[\_0-9a-zA-Z]+\:?"
        
    lines = input_file
    num_lines = len(input_file)
    ret = []
    for text in tqdm(lines):
        text = re.sub(url_match, u"", text)
        text = re.sub(name_match, u"", text)
        text = re.sub("\&amp\;?", u"", text)
        text = re.sub("[\:\.]{1,}$", u"", text)
        text = re.sub("^RT\:?", u"", text)
        text = u''.join(text)
        text = text.strip()
        ret.append(text)
    return ret

In [10]:
df_proc = process_raw_data(df)

In [12]:
#toquenizado
def tokenize_sentences(sentences):
    ret = []
    max_s = len(sentences)
    print("Got " + str(max_s) + " sentences.")
    for s in tqdm(sentences):
        tokens = []
        words = re.split(r'(\s+)', s)
        if len(words) > 0:
            for w in words:
                if w is not None:
                    w = w.strip()
                    w = w.lower()
                    w = ''.join(c for c in w if c not in punctuation)
                    if w.isspace() or w == "\n" or w == "\r":
                        w = None
                    if len(w) < 1:
                        w = None
                    if w is not None:
                        tokens.append(w)
        if len(tokens) > 0:
            ret.append(tokens)
    return ret

In [13]:
df_proc_T = tokenize_sentences(df_proc)

Got 45709 sentences.


In [14]:
#limpieza de los tokens
def clean_sentences(tokens):
    ret = []
    max_s = len(tokens)
    for sentence in tqdm(tokens):
        cleaned = []
        for token in sentence:
            if len(token) > 0:
                if cachedStopWords is not None:
                    for s in cachedStopWords:
                        if token == s:
                            token = None
                if token is not None:
                    if re.search("^[0-9\.\-\s\/]+$", token):
                        token = None
                if token is not None:
                    cleaned.append(token)
        if len(cleaned) > 0:
            ret.append(cleaned)
    return ret

In [15]:
df_proc_T_C = clean_sentences(df_proc_T)
df_proc_T_C[:3]

[['jo', 'també', 'però', 'diria', 'reflexiu', 'és', 'sobrer'],
 ['fuerte', 'lastima'],
 ['bro',
  'quejo',
  'si',
  'parece',
  'puta',
  'madre',
  'mejores',
  'casters',
  'lvp',
  'solo',
  'comentaba']]

In [16]:
#generacion de un vocabulario con frecuencias
def get_word_frequencies(corpus):
    frequencies = Counter()
    for sentence in tqdm(corpus):
        for word in sentence:
            frequencies[word] += 1
    freq = frequencies.most_common()
    return freq

In [17]:
df_proc_T_C_voc = get_word_frequencies(df_proc_T_C)
df_proc_T_C_voc[:10],len(df_proc_T_C_voc)

([('si', 3819),
  ('barcelona', 1914),
  ('q', 1452),
  ('gracias', 1398),
  ('ser', 1339),
  ('día', 1162),
  ('ver', 1139),
  ('mejor', 1114),
  ('hoy', 1105),
  ('bien', 1082)],
 65357)

## entrenamiento del modelo

In [18]:
#size: Nª of layers
#window: Maximum distance between the current and predicted word within a sentence
#min_count: nº of times a word repeats to be taken into account
#sg: training algorithm. 0=cbow, 1=skip-gram
#workers: nº of cores running. no cyton installed--> no paralelization

model = gensim.models.Word2Vec(size=200, window=8, min_count=5, sg=1, workers=multiprocessing.cpu_count())
model.build_vocab(df_proc_T_C)  # prepare the model vocabulary

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [19]:
#nº de palabras unicas en nuestro vocabulario
len(model.wv.vocab)

10941

In [20]:
def create_embeddings(word2vec):
    all_word_vectors_matrix = word2vec.wv.syn0
    num_words = len(all_word_vectors_matrix)
    vocab = word2vec.wv.vocab.keys()
    vocab_len = len(vocab)
    dim = word2vec.wv[vocab[0]].shape[0]
    embedding = np.empty((num_words, dim), dtype=np.float32)
    metadata = ""
    for i, word in enumerate(vocab):
        embedding[i] = word2vec.wv[word]
        metadata += word + "\n"
    metadata_file = os.path.join(save_dir, "metadata.tsv")
    with io.open(metadata_file, "w", encoding="utf-8") as f:
        f.write(metadata)
 
    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    X = tf.Variable([0.0], name='embedding')
    place = tf.placeholder(tf.float32, shape=embedding.shape)
    set_x = tf.assign(X, place, validate_shape=False)
    sess.run(tf.global_variables_initializer())
    sess.run(set_x, feed_dict={place: embedding})
 
    summary_writer = tf.summary.FileWriter(save_dir, sess.graph)
    config = projector.ProjectorConfig()
    embedding_conf = config.embeddings.add()
    embedding_conf.tensor_name = 'embedding:0'
    embedding_conf.metadata_path = 'metadata.tsv'
    projector.visualize_embeddings(summary_writer, config)
 
    save_file = os.path.join(save_dir, "model.ckpt")
    print("Saving session...")
    saver = tf.train.Saver()
    saver.save(sess, save_file)

In [ ]:
!tensorboard --logdir foo:C:\Users\alonsoga\

# fin de construccion

--------------

# unificacion de tema de tweets

para cada usuario, tomamos un corpus con sus tweets. aplicamos TF/IDF a los tokens de su corpus y, con cada indicador, realizamos la media ponderada de los vectores. Obtenemos un espacio vectorial que refleja el tema del usuario.



mas adelante, restaremos los espacios vectoriales entre los usuarios para saber la similitud de los usuarios

In [ ]:
corpus["428308122"][:10]

TF = (Nº de veces que una palabra aparece en un documento)/(Nº de palabras en un documento)

IDF = log(Nº total de documentos/Nº de documentos con el token)

In [ ]:
def tf_idf(corpus):
    #nº de documentos
    n_docs = len(corpus.keys())

    #lista de tokens
    l_tokens = []
    
    #dic frecuencia de cada token
    frecuencias= {}

    #dic de cada token con el nº de documentos con ese token
    tot_frecuency = {}       

    for users in corpus.keys():
        for text in corpus[users]:
            l_tokens.append(text)
            if text in frecuencias:
                frecuencias[text] +=1
            else:
                frecuencias[text] = 1

        for word in set(l_tokens):
            if word in tot_frecuency:
                if word in corpus[users]:
                    tot_frecuency[word] += 1
            else:
                tot_frecuency[word] = 1


    #dic con ratio de aparicion de cada token sobre el total de tokens
    tf = {}
    for i in frecuencias:
        tf[i] = frecuencias[i]/len(l_tokens)

    idf = {}
    for j in tot_frecuency.keys():
        idf[j] = np.log(n_docs/tot_frecuency[j])
    
    
    tf_idf = {}
    for k in tf.keys():
        tf_idf[k] = tf[k]*idf[k]
    
    
    
    return tf_idf

In [ ]:
#ojito cuidao que tarda en ejecutarse 10-15 minutos!

corpus_tf_idf = tf_idf(corpus)

In [ ]:
#valor tf_idf de cada palabra del vocabulario
print(list(corpus_tf_idf.keys())[:5],[corpus_tf_idf[i] for i in list(corpus_tf_idf.keys())[:5]])

con el peso tf_idf de cada palabra, ponderamos los pesos de los vectores de las palabras de los tweets de un usuario para obtener un vector representativo del usuario.

In [ ]:
v_corpus = {}

for key in corpus.keys():
    vector = np.array([0]*100, dtype="float64")
    contador = 0
    for word in corpus[key]:
        if word in model.wv.vocab:
            vector += model.wv.word_vec(word)*corpus_tf_idf[word]
            contador +=1
    v_corpus[key] = vector/contador

In [ ]:
#vector de tema por usuario
v_corpus

analizamos el coeficiente de correlacion vectorial de los usuarios entre si

In [ ]:
data = pd.DataFrame(v_corpus)
data_T = data.transpose(copy=True)
data_T.head()

Realizamos una matriz de correlacion de pearson entre los usuarios

In [ ]:
corrmat = data.iloc[:10].corr() 
  
f, ax = plt.subplots(figsize =(20, 20)) 
sns.heatmap(corrmat, ax = ax, cmap ="YlGnBu", linewidths = 0.1) 

quizas esto no tenga sentido. por qué? porque aunque sean variables continuas, que el vector de un usuario correlacione con otro solo significa que hablan de temas que se distribuyen linealmente de la misma manera. 

ahora, la correlacion entre vectores podria indicar algo??

In [ ]:
corrmat = data_T.corr() 
  
f, ax = plt.subplots(figsize =(15, 15)) 
sns.heatmap(corrmat, ax = ax, cmap ="YlGnBu", linewidths = 0.1) 

no


quizas podemos ver la cercania de los usuarios en base a la cercania de sus vectores.  
1) **distancia euclidea** entre los vectores entre pares de usuarios: siendo 0 la igualdad y 1 la diferencia extrema

In [ ]:
#ojo que la ejecucion es hora y media!!
euc_corpus = pd.DataFrame(columns=v_corpus.keys(), index=v_corpus.keys(), data=0, dtype="float32")

In [ ]:
for i in tqdm_notebook(v_corpus.keys()):
    for j in v_corpus.keys():
        if i != j and not np.isnan(v_corpus[i]).any() and not np.isnan(v_corpus[j]).any():
            euc_corpus.loc[i][j] = distance.euclidean(v_corpus[i], v_corpus[j])

In [ ]:
euc_corpus.head()

In [ ]:
euc_corpus.to_csv("./data/euc_corpus.txt", sep="&")

In [ ]:
euc_corpus = pd.read_csv("./data/euc_corpus.txt", sep="&").head()

In [ ]:
tabla = list(euclidean.keys())[1:100]
euc_corpus_20 = pd.DataFrame(index=tabla, columns=tabla, data = euc_corpus)

In [ ]:
f, ax = plt.subplots(figsize =(15, 15)) 
sns.heatmap(euc_corpus_20, ax = ax, cmap ="YlGnBu", linewidths = 0.1)

2) **distancia de mahalanobis**: se implementaría así, pero el tiempo innecesario y nuestro criterio, hace innecesario procesarlo. Queda a modo de ejemplo

In [ ]:
#ojo que la ejecucion es... tambien hora y media!!
maha_corpus= pd.DataFrame(columns=v_corpus.keys(), index=v_corpus.keys(), data=0, dtype="float32")

for i in tqdm_notebook(v_corpus.keys()):
    for j in v_corpus.keys():
        if i != j and not np.isnan(v_corpus[i]).any() and not np.isnan(v_corpus[j]).any():
            maha_corpus.loc[i][j] = dst = distance.mahalanobis(v_corpus[i], v_corpus[j], np.linalg.inv(np.cov(pd.DataFrame(v_corpus).T)))

http://dataaspirant.com/2015/04/11/five-most-popular-similarity-measures-implementation-in-python/


https://www.machinelearningplus.com/statistics/mahalanobis-distance/




conclusion: aunque la distancia euclidea es una medida de comparacion ampliamente usada, tenemos que tener en cuenta nuestros datos para elegir la mejor. En este caso, los vectores, aunque son continuos, representan informacion categorica. Por ejemplo, en una dimension concreta de un vector, el doble de un valor no representa dos veces la unidad inicial. perro = 1, elefante =1.1


siendo así, nos decantamos por la distancia de Mahalonobis o incluso la distancia de Bhattacharyya/Cavalli-Sforza

3) **Distancia Bhattacharyya / Cavalli-Sforza**

In [ ]:
import math

def bhattacharyya(a, b):

    return -math.log(sum([np.sqrt(abs(np.multiply(u,v)), dtype="float32") for u,v in zip(list(a),list(b))]))

In [ ]:
#ojo que la ejecucion es hora y media!!
bha_corpus = pd.DataFrame(columns=v_corpus.keys(), index=v_corpus.keys(), data=0, dtype="float32")

In [ ]:
for i in tqdm_notebook(v_corpus.keys()):
    for j in v_corpus.keys():
        if i != j and not np.isnan(v_corpus[i]).any() and not np.isnan(v_corpus[j]).any():
            bha_corpus.loc[i][j] = bhattacharyya(v_corpus[i], v_corpus[j])

In [ ]:
bha_corpus.to_csv("./data/bha_corpus.txt", sep="&")

In [ ]:
bha_corpus = pd.read_csv("./data/bha_corpus.txt", sep="&", index_col=1).drop(bha_corpus.columns[0], axis=1)

In [ ]:
bha_corpus

In [ ]:
tabla = list(bha_corpus.keys())[1:100]
bha_corpus_20 = pd.DataFrame(index=tabla, columns=tabla, data = bha_corpus)

In [ ]:
f, ax = plt.subplots(figsize =(15, 15)) 
sns.heatmap(bha_corpus_20, ax = ax, cmap ="YlGnBu", linewidths = 0.1)